# A3

## Packages
### Used Packages 
* [tslearn](https://tslearn.readthedocs.io/en/stable/quickstart.html)

### Not currently used packages, but might use
* [DBA: Dynamic Time Warping Barycenter Averaging](https://github.com/fpetitjean/DBA)

## References
* [OpenFace 2.0](https://github.com/TadasBaltrusaitis/OpenFace/wiki): Facial Behavior Analysis Toolkit Tadas Baltrušaitis, Amir Zadeh, Yao Chong Lim, and Louis-Philippe Morency, IEEE International Conference on Automatic Face and Gesture Recognition, 2018
* Cassisi, C., Montalto, P., Aliotta, M., Cannata, A., & Pulvirenti, A. (2012). _Similarity measures and dimensionality reduction techniques for time series data mining_. Advances in data mining knowledge discovery and applications, 71-96.
* McKinney, W. (2012). _Python for data analysis: Data wrangling with Pandas, NumPy, and IPython_. " O'Reilly Media, Inc.".
* Shokoohi-Yekta, M., Hu, B., Jin, H., Wang, J., & Keogh, E. _Generalizing Dynamic Time Warping to the Multi-Dimensional Case Requires an Adaptive Approach (SDM 2015)_. In 2015 SIAM International Conference on Data Mining.-http://www. cs. ucr. edu/~ eamonn/Multi-Dimensional_DTW_Journal. pdf (last access: 18.12. 2015).
* [TsLearn](https://tslearn.readthedocs.io/en/stable/gettingstarted.html) documentation


In [89]:
%ls

A3.ipynb               GIFGIF_Download.ipynb  utilities/
CreateDataset.ipynb    requirements.txt       venv/
GIFGIF_Dataset/        run_openface.py


## Imports

In [90]:
import pandas
import glob
from tslearn.utils import to_time_series, to_time_series_dataset

# Regenerate all data
set `REGENERATE` to true if you want to regenerate all data. If set to false, data saved to disk will be used. Regenerating all the data may be time-consuming, so be warned.

In [91]:
REGENERATE = True

## Dataset creation

As I am going through my semester working through 4 higher-division classes and 20 hours of part-time work per week, my emotions have shifted from peace to a combination of sadness, anger and powerlessness. In order to keep my attitude positive, I chose the better of the three: sadness. 

I was looking to run OpenFace on the gifs, but that is apparently not possible. After spending enough hours trying to install OpenFace on my machine through 3 different methods given in the Wiki (native MacOS, `bamos/openface` docker image, `algebr/openface` docker image and creating my own docker version), I decided to go with the existing docker image from `algebr/openface`, using the method [described here](https://github.com/TadasBaltrusaitis/OpenFace/wiki). I had issues with all of the other methods.

I converted all gifs to images, copied them to the docker container and and ran OpenFace on the sequences of images.
```bash=
docker run -it -d --rm algebr/openface:latest
docker cp images 3a73fbce562e:/home/openface-build
docker exec -it 3a73fbce562e sh
docker cp run_openface.py 3a73fbce562e:/home/openface-build
```
I then ran my `run_openface.py` script to create the CSVs out of the image folders, and copied the results to my local machine:
```bash
docker cp  40b27:/home/openface-build/output ./output
```

My first look at the processed data seem to imply that OpenFace failed to parse all of the cartoons. It did show partial information, but instances of cartoons may need to be dropped from the dataset. Only images of real humans were fully labelled.

_NOTE: not sure how to submite my assignment with my dataset as it is 2GB_

## Load Dataset
Load the generated CSVs into a pandas dataframe.

In [92]:
%%time

OPENFACE_DATA = "GIFGIF_Dataset/openface"
CSV_EXT = ".csv"

dirs = glob.glob(f"{OPENFACE_DATA}/*")

# Save every gif's openface dataset in a pandas dataframe.
time_series_dataset = []

for dir in dirs[:10]:
    filename = dir.split("/")[-1]
    csv_filepath = f"{dir}/{filename}{CSV_EXT}"
    with open(csv_filepath) as f:
        df = pandas.read_csv(f)

        # Column names sometimes have an extra space on the left or right
        df.rename(columns=lambda x: x.strip(), inplace=True)

        # Remove all NA rows
        df.dropna()

        if df.loc[:, "confidence"].mean() > 0:
            formatted_time_series = to_time_series(df)
            time_series_dataset.append(formatted_time_series)

X_train = to_time_series_dataset(time_series_dataset)
len(X_train)

CPU times: user 501 ms, sys: 22.7 ms, total: 524 ms
Wall time: 544 ms


4

## Approach

I am not certain how to approach a multivariate DTW problem. It seems like there are two main approaches [(2015, Shokoohi-Yekta et al)](https://link.springer.com/article/10.1007/s10618-016-0455-0):
1. Independent DTW: find the distance between every dimension between two time series. For multi-dimensional time series (MDT) Q and C, $$DTW_1(Q, C) = \sum_{m=1}^{M} DTW(Q_m, C_m)$$ Each dimension is independent.
2. Dependent DTW: warp all dimensions in a single warping matrix. The dimensions are now dependent. It's similar to single-dimension DTW, except the distance is measured for M data-points. The following distance function is used: $$d(q_i, c_j) = \sum_{m=1}^{M} DTW(q_{i, m}, c_{j, m})^2$$

### Areas of interest in the data
[See here for in-depth explanations about output format of OpenFace](https://github.com/TadasBaltrusaitis/OpenFace/wiki/Output-Format)

[Action Units information](https://github.com/TadasBaltrusaitis/OpenFace/wiki/Action-Units)

#### All columns in the CSVs



| Column                         | Description                                                  | Notes                                                        | Selected |
| ------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ | -------- |
| `gaze_0_x, gaze_0_y, gaze_0_z` | eye 0, leftmost eye in image                                 |                                                              | N        |
| `gaze_1_x, gaze_1_y, gaze_1_z` | Eye 1, rightmost eye in image                                |                                                              | N        |
| `gaze_angle_x, gaze_angle_y`   | Eye gaze direction in radians in world coordinates averaged for both eyes and converted into more easy to use format than gaze vectors | Might want to use that over `gaze`, as `gaze` is containted in it. | Y        |
| `eye_lmk_x*, eye_lmk_X*`       | location of 3D eye in pixels (x) and mm (X)                  | Redundant if using head rotation and translation             | N        |
| `pose_Rx, pose_Ry, pose_Rz`    | pitch, yaw and roll for head in radians in relation to camera | Head rotation may be useful                                  | Y        |
| `pose_Tx, pose_Ty, pose_Tz`    | location of the head with respect to camera in millimeters   | Head translation may be useful                               | Y        |
| `AU_r`                         | Action units intesity of 17 AUs (from 0 to 5)                | Definitely useful                                            | Y        |
| `AU_c`                         | Action unit presence of 18 AUs                               | Definitely useful                                            | Y        |

#### Fourier Transform
Should I apply a Fourier transform on the time series, and run DTW on the output, or should I simply run DTW on the current time series?

#### Number of features
The model may overfit if too many features are provided. Try at least 3 different amount of features.

## Preprocessing
* Normalize to have zero mean and unit variance
* [Resample?](https://tslearn.readthedocs.io/en/stable/gen_modules/preprocessing/tslearn.preprocessing.TimeSeriesResampler.html#tslearn.preprocessing.TimeSeriesResampler)